In [1]:
import random
import string
import re
from time import sleep
import os
import openai

def make_random_string():
    length = random.randint(1, 30)
    random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=length))
    return random_string

test_bank_size = 20

# should_contain_strings = [make_random_string() for _ in range(test_bank_size)]
# non_contain_strings = [make_random_string() for _ in range(test_bank_size)]

random_strings = [make_random_string() for _ in range(test_bank_size)]
should_contain_strings = [s for s in random_strings if 'i' in s]
non_contain_strings = [s for s in random_strings if 'i' not in s]


first_message = """
Your job is to write a regular expression. I will evaluate your regular expression against two lists of strings (which I am not sharing).

The first list of strings is the list we want the regex to match. The second list is the list we want not to match.

When you supply a regex, I will evaluate it against both lists and tell you how many of each list it matched.

Then your job will be to try again with another regular expression to see if you can get a better score (ideally improving this with experimentation).

Please reply with only the regex you want me to test.
"""

def pull_regex_from_response(res):
    regex = res.choices[0].message.content
    return regex

def evaluate_regex(regex):
    should_contain_matches = [s for s in should_contain_strings if re.search(regex, s)]
    non_contain_matches = [s for s in non_contain_strings if re.search(regex, s)]
    return len(should_contain_matches), len(non_contain_matches)

def next_user_message(good_match, bad_match):
    out = f"""This regex correctly matched {good_match} of the {test_bank_size} strings it should match.
It also incorrectly matched {bad_match} of the {test_bank_size} strings it should not match.

The numerical score for this regex is {good_match - bad_match}.

Please reply with the regex you want me to test next (and no other text)."""
    return out


openai.api_key = os.getenv("OPENAI_API_KEY")

def make_api_call(messages):
   response = openai.ChatCompletion.create(model="gpt-4",
                                           messages=messages
                                           )
   return response

messages = [{'role': 'user', 'content': first_message}]
for i in range(15):
    api_response = make_api_call(messages)
    regex = pull_regex_from_response(api_response)
    good_match, bad_match = evaluate_regex(regex)
    messages.append({'role': 'assistant', 'content': regex})
    messages.append({'role': 'user', 'content': next_user_message(good_match, bad_match)})
    print(f"Regex: {regex}    Score: {good_match - bad_match}")
    sleep(2)


AttributeError: module 'openai' has no attribute 'ChatCompletion'